In [1]:
import requests
from bs4 import BeautifulSoup
import config #api_key
import functions
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

# Data Aggregation

In [2]:
artists = ['NAV', 'Future' ,'Lil Uzi Vert', 'Meek Mill', 'J. Cole', 'Chief Keef', \
           'Young Thug', 'Travis Scott', '21 Savage', 'Gunna' ,'Lil Baby',\
           'Famous Dex', 'The Weeknd' ,'Juice WRLD', 'Offset', 'Migos', 'Trippie Redd']

In [47]:
def productionize(name):
    song_lyrics = []
    
    artist = functions.request_artist(name)
    artist_id = functions.get_artist_id(artist)
    songs = functions.get_song_id(artist_id)
    
    songs = [song["id"] for song in songs
        if song["primary_artist"]["id"] == artist_id]
    
    for song in songs:
        song_lyrics.append(functions.retrieve_lyrics(song))
    
    print('Colleted all lyrics for {}'.format(name))
    return functions.lyr_to_df(song_lyrics, name)

In [45]:
df_lists = []

In [50]:
for artist in artists:
    df_lists.append(productionize(artist))
# productionize('Travis Scott')

Page 2 finished scraping
Page 3 finished scraping
Page 4 finished scraping
Page 5 finished scraping
Page 6 finished scraping
Page 7 finished scraping
Page 8 finished scraping
Page 9 finished scraping
Page 10 finished scraping
Page 11 finished scraping
Page 12 finished scraping
Collected lyrics for all songs
Colleted all lyrics for NAV
Page 2 finished scraping
Page 3 finished scraping
Page 4 finished scraping
Page 5 finished scraping
Page 6 finished scraping
Page 7 finished scraping
Page 8 finished scraping
Page 9 finished scraping
Page 10 finished scraping
Page 11 finished scraping
Page 12 finished scraping
Page 13 finished scraping
Page 14 finished scraping
Page 15 finished scraping
Page 16 finished scraping
Page 17 finished scraping
Page 18 finished scraping
Page 19 finished scraping
Page 20 finished scraping
Page 21 finished scraping
Page 22 finished scraping
Page 23 finished scraping
Page 24 finished scraping
Page 25 finished scraping
Page 26 finished scraping
Page 27 finished scra

Page 5 finished scraping
Page 6 finished scraping
Page 7 finished scraping
Page 8 finished scraping
Page 9 finished scraping
Page 10 finished scraping
Page 11 finished scraping
Page 12 finished scraping
Page 13 finished scraping
Page 14 finished scraping
Page 15 finished scraping
Page 16 finished scraping
Page 17 finished scraping
Page 18 finished scraping
Page 19 finished scraping
Page 20 finished scraping
Page 21 finished scraping
Page 22 finished scraping
Page 23 finished scraping
Page 24 finished scraping
Page 25 finished scraping
Page 26 finished scraping
Page 27 finished scraping
Collected lyrics for all songs
Colleted all lyrics for Famous Dex
Page 2 finished scraping
Page 3 finished scraping
Page 4 finished scraping
Page 5 finished scraping
Page 6 finished scraping
Page 7 finished scraping
Page 8 finished scraping
Page 9 finished scraping
Page 10 finished scraping
Page 11 finished scraping
Page 12 finished scraping
Page 13 finished scraping
Page 14 finished scraping
Page 15 fin

In [66]:
major_df = pd.concat(df_lists, axis=0)

In [71]:
major_df.reset_index(drop=True, inplace=True)

In [73]:
lyrics_tuples = [tuple(x) for x in major_df.to_records(index=False)]

Connecting to DB

In [74]:
cnx = mysql.connector.connect(
      host = config.my_cred['host'],
      user = config.my_cred['user'],
      passwd = config.my_cred['pw'],
      database= 'genius_lyrics'
)

cursor = cnx.cursor()

Create DB

In [18]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [19]:
create_database(cursor, 'genius_lyrics')

In [38]:
#CREATE TABLE
cursor.execute("""USE genius_lyrics""")
TABLES = {}
TABLES['lyrics'] = (
    "CREATE TABLE lyrics ("
    "  id INT NOT NULL AUTO_INCREMENT,"
    "  artist varchar(255),"
    "  titles varchar(255),"
    "  lyric TEXT NOT NULL,"
    "  PRIMARY KEY (id)"
    ") ENGINE=InnoDB")

In [39]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table lyrics: OK


Insert Data Into Table

In [37]:
def db_insert_lyrics(cnx, cursor, lyrics):
    stmt = ("INSERT INTO lyrics (artist, titles, lyric) VALUES (%s,%s, %s)")
    cursor.executemany(stmt, lyrics)
    cnx.commit()

In [75]:
db_insert_lyrics(cnx, cursor, lyrics_tuples)
cnx.close()